<a href="https://colab.research.google.com/github/Pamela2710/Data-Mining/blob/main/DataProcessingClassification/DataProcessing_and_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel, mutual_info_classif
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.base import clone
from scipy.stats import entropy
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest

df = pd.read_csv('https://raw.githubusercontent.com/Pamela2710/Data-Mining/main/DataProcessing/features_completas.csv', encoding='latin-1')

# Eliminar filas con valores NaN
df = df.dropna()

df

,Media tiempo,Desviacion estandar tiempo,Varianza tiempo,Entropia tiempo,kurtosis tiempo,MSE tiempo,Tiempo en alcanzar el pico maximo en frecuencia,RMS tiempo,Peak2peak tiempo,TIEMPO peak2rms,...,solidity,std_dev,x_center_mass,x_centroid,x_centroid_cm_diff,y_center_mass,y_centroid,y_centroid_cm_diff,Etiqueta,espectrograma etiqueta
0,0.099520,0.245902,0.108890,0,0.013447,0.907095,0.115646,0.245920,0.164061,0.110557,...,1,0.410795,0.406519,0.405904,-0.000615,0.472183,0.471469,-0.000713,Long Period,espect_event_1
1,0.085819,0.876726,0.796870,0,0.050793,0.351758,0.306122,0.876697,1.000000,0.238391,...,1,12.640444,0.240467,0.239852,-0.000615,0.279601,0.278887,-0.000713,Volcano Tectonic,espect_event_2
2,0.077419,0.231336,0.099950,0,0.015484,0.882481,0.122449,0.231357,0.154656,0.130988,...,1,4.451864,0.422509,0.421894,-0.000615,0.490728,0.490014,-0.000713,Long Period,espect_event_3
3,0.102426,0.291445,0.138865,0,0.011577,0.910015,0.102041,0.291448,0.160484,0.092834,...,1,7.681243,0.418819,0.418204,-0.000615,0.486448,0.485735,-0.000713,Long Period,espect_event_4
4,0.091737,0.428982,0.247991,0,0.026587,0.723575,0.108844,0.428999,0.379571,0.141011,...,1,2.908373,0.287208,0.286593,-0.000615,0.333809,0.333096,-0.000713,Hybrid,espect_event_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,0.101867,0.214477,0.089995,0,0.018980,0.875568,0.333333,0.214480,0.163796,0.131218,...,1,2.546444,0.421894,0.421279,-0.000615,0.490014,0.489301,-0.000713,Long Period,espect_event_686
686,0.086898,0.680620,0.520007,0,0.038755,0.298080,0.455782,0.680617,0.504337,0.150002,...,1,15.365171,0.325953,0.325338,-0.000615,0.378745,0.378031,-0.000713,Volcano Tectonic,espect_event_687
687,0.077123,0.548769,0.365809,0,0.024871,0.207060,0.510204,0.548773,0.259238,0.084483,...,1,9.442264,0.306888,0.306273,-0.000615,0.356633,0.355920,-0.000713,Volcano Tectonic,espect_event_688
688,0.079244,0.720172,0.571271,0,0.031290,0.165912,0.551020,0.720172,0.407933,0.102759,...,1,14.023466,0.337638,0.337023,-0.000615,0.392297,0.391583,-0.000713,Volcano Tectonic,espect_event_689


In [5]:
# Extraer la columna "Etiqueta"
etiquetas = df['Etiqueta']

# Normalizar las columnas numéricas
cols_numericas = df.columns[:-3]  # Excluyendo las columnas "Etiqueta" y "espectrograma etiqueta"
scaler = MinMaxScaler()
df[cols_numericas] = scaler.fit_transform(df[cols_numericas])

# Unir la columna "Etiqueta" a los datos normalizados
df_normalized = pd.concat([df[cols_numericas], etiquetas], axis=1)

# Mostrar el resultado
df_normalized

,Media tiempo,Desviacion estandar tiempo,Varianza tiempo,Entropia tiempo,kurtosis tiempo,MSE tiempo,Tiempo en alcanzar el pico maximo en frecuencia,RMS tiempo,Peak2peak tiempo,TIEMPO peak2rms,...,perimeter,skewness,solidity,std_dev,x_center_mass,x_centroid,x_centroid_cm_diff,y_center_mass,y_centroid,Etiqueta
0,0.099520,0.226593,0.102626,0.0,0.013447,0.907095,0.115646,0.226602,0.163998,0.110557,...,0.750000,0.262347,0.0,0.014767,0.503906,0.503906,0.0,0.503906,0.503906,Long Period
1,0.085819,0.873569,0.795442,0.0,0.050793,0.351758,0.306122,0.873538,1.000000,0.238391,...,0.402174,0.009667,0.0,0.499546,0.152344,0.152344,0.0,0.152344,0.152344,Volcano Tectonic
2,0.077419,0.211654,0.093624,0.0,0.015484,0.882481,0.122449,0.211665,0.154592,0.130988,...,0.827640,0.046256,0.0,0.174953,0.537760,0.537760,0.0,0.537760,0.537760,Long Period
3,0.102426,0.273302,0.132812,0.0,0.011577,0.910015,0.102041,0.273296,0.160420,0.092834,...,0.874224,0.024948,0.0,0.302965,0.529948,0.529948,0.0,0.529948,0.529948,Long Period
4,0.091737,0.414361,0.242705,0.0,0.026587,0.723575,0.108844,0.414371,0.379524,0.141011,...,0.647516,0.071298,0.0,0.113770,0.251302,0.251302,0.0,0.251302,0.251302,Hybrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,0.101867,0.194363,0.083598,0.0,0.018980,0.875568,0.333333,0.194357,0.163733,0.131218,...,0.354037,0.062165,0.0,0.099423,0.536458,0.536458,0.0,0.536458,0.536458,Long Period
686,0.086898,0.672442,0.516633,0.0,0.038755,0.298080,0.455782,0.672435,0.504299,0.150002,...,0.583851,0.012588,0.0,0.607553,0.333333,0.333333,0.0,0.333333,0.333333,Volcano Tectonic
687,0.077123,0.537215,0.361351,0.0,0.024871,0.207060,0.510204,0.537213,0.259182,0.084483,...,0.591615,0.020329,0.0,0.372771,0.292969,0.292969,0.0,0.292969,0.292969,Volcano Tectonic
688,0.079244,0.713007,0.568258,0.0,0.031290,0.165912,0.551020,0.713004,0.407889,0.102759,...,0.482919,0.014551,0.0,0.554368,0.358073,0.358073,0.0,0.358073,0.358073,Volcano Tectonic


In [6]:
X_normalized = scaler.fit_transform(df_normalized.drop(['Etiqueta'], axis=1))
y = df_normalized['Etiqueta']

# 'X_normalized' es el conjunto de datos normalizado y 'y' son las etiquetas
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Implementación de la función objetivo GR
def gain_ratio(X_subset, y, ):
    # Utiliza SelectKBest y mutual_info_classif para calcular el Gain Ratio
    scores = mutual_info_classif(X_subset, y)

    # Calcula el índice de ganancia (o información) para cada característica
    gain_ratio_per_feature = scores / np.sum(scores)
    print("Indice de ganancia por caracteristica: ", gain_ratio_per_feature)

    # Retorna tanto el índice de ganancia para cada característica como el promedio
    return np.mean(scores)


# Implementación ABC
def artificial_bee_colony(X, y, num_iterations, num_employed_bees, num_onlooker_bees):
    num_features = X.shape[1]
    results = []

    for _ in range(num_iterations):
        print('Iteracion #: ', _)
        # Randomly generate employed bees' solutions
        solutions = np.random.choice([0, 1], size=(num_employed_bees, num_features))

        # Evaluate the solutions
        scores = np.zeros(num_employed_bees)
        for i in range(num_employed_bees):
            selected_features = np.where(solutions[i] == 1)[0]
            X_subset = X[:, selected_features]
            score = gain_ratio(X_subset, y)  # Utiliza la función objetivo GR
            scores[i] = score
            # Guarda los resultados
            results.append((selected_features, score))

        # Update solutions based on scores
        for i in range(num_onlooker_bees):
            bee_index = np.random.randint(num_employed_bees)
            if np.random.rand() < scores[bee_index] / np.max(scores):
                solutions[bee_index] = np.random.choice([0, 1], size=num_features)

    # Ordena los resultados por importancia
    results.sort(key=lambda x: x[1], reverse=True)

    return results[:5]  # Devuelve los cinco mejores subconjuntos

# Ejemplo de ejecución
num_iterations = 250
num_employed_bees = 100
num_onlooker_bees = 20

top_five_subsets = artificial_bee_colony(X_train, y_train, num_iterations, num_employed_bees, num_onlooker_bees)

# Imprime los cinco mejores subconjuntos y su importancia per cápita
for i, (subset, importance) in enumerate(top_five_subsets):
    print(f"Top {i+1} Subset: {subset}, Importance: {importance}")
    print('Size: ', len(subset))

    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_train[:, subset], y_train)
    y_pred = clf.predict(X_test[:, subset])
    auc_score = accuracy_score(y_test, y_pred)
    print("AUC Score:", auc_score)

Se truncaron las últimas líneas 5000 del resultado de transmisión.
 0.00149282 0.01300942 0.00097415 0.02255467 0.02144807 0.03208449
 0.05361021 0.01268777 0.01538312 0.00768452 0.00408228 0.01095047
 0.         0.0039043  0.0017562  0.03897103 0.01155605 0.05577802
 0.         0.03396761 0.03180478 0.00888653 0.00386219 0.04274391
 0.00524956 0.00903729 0.0454858  0.00422949 0.01864995 0.04332926
 0.00889326 0.05260705 0.053031   0.         0.053425   0.05208652]
Indice de ganancia por caracteristica:  [5.89648395e-03 1.39658172e-02 1.77533914e-02 1.93959109e-02
 1.92807857e-03 2.26407454e-02 2.96986156e-04 1.09731293e-02
 1.93983326e-02 0.00000000e+00 1.14938854e-02 3.14169026e-02
 2.18192182e-02 2.51388956e-02 4.46385040e-02 1.92071578e-03
 2.47510555e-02 1.00401405e-02 5.66959302e-03 4.19983602e-03
 6.68575788e-04 2.62962939e-02 2.54469779e-02 3.80568466e-02
 6.39877914e-02 5.34691639e-03 3.12292436e-05 1.83627812e-02
 5.52514335e-03 4.82993600e-02 2.95957239e-02 8.48891885e-03
 5

In [12]:
import os
subset_data = []

# Recorre los cinco mejores subconjuntos
for i, (subset, importance) in enumerate(top_five_subsets):
    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_train[:, subset], y_train)
    y_pred = clf.predict(X_test[:, subset])
    auc_score = accuracy_score(y_test, y_pred)

    # Agrega los datos del subconjunto a la lista
    subset_data.append({'Top Subset': subset, 'Importance': importance, 'AUC Score': auc_score})

# num_iterations = 250
# num_employed_bees = 100
# num_onlooker_bees = 20

# Crea un DataFrame a partir de los datos de los subconjuntos
df = pd.DataFrame(subset_data)

df.to_csv('/content/drive/MyDrive/TopSubsets/top_five_subsets1.csv', index=False)